In [4]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.metrics import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM
from tensorflow.keras.optimizers import SGD
from keras.utils.vis_utils import plot_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import *

import numpy.random as rand
import scipy.stats as stats 
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor 
from sklearn.model_selection import StratifiedKFold

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, timedelta, time, datetime
import time
plt.style.use('ggplot')

Use root mean squared log error

Pipeline?

train = 3 or 4 weeks, test = 1 day / week

## Train/Test Data

Creating train/test data

train_test_split(y, shuffle=False) instead of manually selecting train/test data?

or df2 = datasX.iloc[:, :72]
   df2 = datasX.iloc[:, 72:]

Train = weeks 5/10/21 to 5/31/21

Test = week 6/7/21 + 6/14/21 day 

# converting days to max and min for neural network prediction

In [50]:
week_start_lst = ['2021-05-10', '2021-05-17', '2021-05-24', '2021-05-31', '2021-06-7']
week_start_df = pd.DataFrame(columns=['year', 'month', 'day'])
for i, week in enumerate(week_start_lst):
    split_week = week.split('-')
    week_start_df.loc[i] = split_week[0], split_week[1], split_week[2]
week_start_datetime = pd.to_datetime(week_start_df)

In [55]:
for week_start in week_start_datetime:
    for day_num in range(2):
        trading_day = week_start + timedelta(days = day_num)
        trading_day = trading_day.date()
        cptv_trading_day_path = f'closing_price_trade_volume_data/AMZN_cptv_{trading_day}.xls'
        cptv_trading_day_df = pd.read_excel(cptv_trading_day_path)
        print(cptv_trading_day_df.head(5))

               Date       Open       High        Low      Close   Volume
0  2021-05-10 09-30  3273.1101  3273.1101  3273.1101  3273.1101  98924.0
1  2021-05-10 09-31  3269.8450  3269.8450  3269.8450  3269.8450  21831.0
2  2021-05-10 09-32  3259.7500  3259.7500  3259.7500  3259.7500  30404.0
3  2021-05-10 09-33  3259.0000  3259.0000  3259.0000  3259.0000  45721.0
4  2021-05-10 09-34  3258.0000  3258.0000  3258.0000  3258.0000  32843.0
               Date       Open       High        Low      Close    Volume
0  2021-05-11 09-30  3145.9099  3145.9099  3145.9099  3145.9099  278087.0
1  2021-05-11 09-31  3148.0601  3148.0601  3148.0601  3148.0601   55566.0
2  2021-05-11 09-32  3152.2649  3152.2649  3152.2649  3152.2649   49978.0
3  2021-05-11 09-33  3144.8523  3144.8523  3144.8523  3144.8523   34608.0
4  2021-05-11 09-34  3144.6799  3144.6799  3144.6799  3144.6799   30181.0
               Date       Open       High        Low      Close    Volume
0  2021-05-17 09-30  3254.7451  3254.7451  3

In [ ]:
def convert_week_to_min_max(week_start_datetime)
    for week_start in week_start_datetime
        for day_num in range(5):
            trading_day = week_start + timedelta(days = day_num)
    
            cptv_trading_day_path = f'closing_price_trade_volume_data/AMZN_cptv_{trading_day}.xls'
            cptv_trading_day_df = pd.read_excel(cptv_trading_day_path)
            cptv_trading_day_df.drop(columns=['Open', 'High', 'Low'], inplace=True)
            cptv_trading_day_df.drop(cptv_trading_day_df.tail(1).index, inplace=True)
            cptv_week_df = cptv_week_df.append(cptv_trading_day_df, ignore_index=True, sort=False)

# methods for decomposition for regression prediction

In [28]:
train = pd.DataFrame()

In [35]:
week_start = '2021-05-31'

In [36]:
cptv_week_csv = f'AMZN_cptv_{week_start}.csv'
cptv_week_df = pd.read_csv(f'C:/Users/seant/stock_analyzer/weekly_data/{cptv_week_csv}', header=0, index_col=0)
train = train.append(cptv_week_df, ignore_index=True, sort=False)

In [ ]:
test_week_start = '2021-06-07'

In [25]:
cptv_week_csv = f'AMZN_cptv_{test_week_start}.csv'
test = pd.read_csv(f'C:/Users/seant/stock_analyzer/weekly_data/{cptv_week_csv}', header=0, index_col=0)

In [57]:
testday = pd.read_excel(f'C:/Users/seant/stock_analyzer/closing_price_trade_volume_data/AMZN_cptv_2021-06-14.xls', header=0, index_col=0)
test = test.append(testday, ignore_index=True, sort=False)

In [58]:
train_csv = f'AMZN_cptv_train.csv'
train.to_csv(path_or_buf=f'C:/Users/seant/stock_analyzer/weekly_data/{train_csv}')
test_csv = f'AMZN_cptv_test.csv'
test.to_csv(path_or_buf=f'C:/Users/seant/stock_analyzer/weekly_data/{test_csv}')

Massaging data

In [53]:
def roll(df, column_name_lst, minute_period=200):
    rolled_df = pd.DataFrame()
    for column_name in column_name_lst:
        series = df.loc[:, column_name]
        rolled_df[column_name] = series.rolling(minute_period, center=True).mean()
    return rolled_df

In [54]:
def decompose(df, column_name_lst, minute_period=390):
    df_copy = df.copy()
    seasonal_resid_df = pd.DataFrame()
    trend_df = pd.DataFrame()
    for column_name in column_name_lst:
        for_decompose = df_copy[[column_name]].dropna()
        decomposed = seasonal_decompose(for_decompose, model='additive', period=minute_period)
        trend_df[column_name] = decomposed.trend.dropna()
        seasonal_resid_df[f'{column_name}_seasonal'] = decomposed.seasonal
        seasonal_resid_df[f'{column_name}_resid'] = decomposed.resid
    #add a column of constants?
    return trend_df, seasonal_resid_df

In [ ]:
def split(trend_df, minute_period=390, num_periods = 5):
    num_prediction_periods = minute_period * num_periods
    last_period_cut = trend_df.shape(0) - num_prediction_periods #want to use 5 periods of data to predict next period of close prices
    X = trend_df.iloc[:, :Last_period_cut] #cuts last period off for features to predict from
    y = trend_df.iloc[0, num_prediction_periods:] #cuts first column and period off for targets for prediction
    #how to use 1 week of data to predict 1 day of prices? extrapolate trend out 1 period?
    return X, y

In [20]:
def recompose(trend_df, seasonal_resid_df, column_name_lst, minute_period=390):
    trend_list = []
    for i in range(minute_periods / 2):
        trend_list.append(np.nan)
    for value in trend_df.items():
        trend_list.append(value)
    for i in range(minute_periods / 2):
        trend_list.append(np.nan)

In [72]:
nan_df = pd.Series(np.nan, index = range(0, int(390 / 2)))
nan_df.isna().sum()

195

rescale data with 

# normalize the dataset for LSTM in neural network

scaler = MinMaxScaler(feature_range=(0, 1))

dataset = scaler.fit_transform(dataset)

Use max/min of each day?

In [41]:
minute_rolling_avg = 200

train_rolled = pd.DataFrame()
train_close_series = train.loc[:, 'Close']
train_rolled = train_rolled.assign(rolling_close = train_close_series.rolling(minute_rolling_avg, center=True).mean())
train_volume_series = train.loc[:, 'Volume']
train_rolled = train_rolled.assign(rolling_volume = train_volume_series.rolling(minute_rolling_avg, center=True).mean())

test_rolled = pd.DataFrame()
test_close_series = test.loc[:, 'Close']
test_rolled = test_rolled.assign(rolling_close = test_close_series.rolling(minute_rolling_avg, center=True).mean())
test_volume_series = test.loc[:, 'Volume']
test_rolled = test_rolled.assign(rolling_volume = test_volume_series.rolling(minute_rolling_avg, center=True).mean())

In [51]:
train_rolled = roll(train, ['Close', 'Volume'])
train_rolled.iloc[500]

Close      3197.335587
Volume    11301.540000
Name: 500, dtype: float64

In [42]:
cptv_copy = cptv_week_df.copy()
cptv_roll_close_for_decompose = cptv_copy[['rolling_close']].dropna()
cptv_roll_vol_for_decompose = cptv_copy[['rolling_volume']].dropna()

(7729, 5)
(1931, 3)


X_train = combine all weeks into one dataframe, decompose into closing(trend_1 + trend_2), volume(trend), +1 column, add additional day as prediction happens to predict next day

y_train = closing(trend_1 + trend_2)? target is next minute price? min, max of next day?

neural network model

In [11]:
model = Sequential()
model.add(LSTM(units=10, return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(LSTM(units=10, return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(LSTM(units=10, return_sequences=True))
model.add(Dropout(rate=0.2))
#model.add(Dense(5, activation='relu'))
model.add(Dense(5))
model.compile(loss='root_mean_squared_error', optimizer='SGD', metrics=keras.metrics.Accuracy())

In [ ]:
batch_size = 300
model.fit(X_train, y_train, epochs=10, batch_size=batch_size)

In [ ]:
results=model.evaluate(X_test, y_test, batch_size=batch_size)

In [ ]:
def buildNN(X, y, activation_function, metric, optimizer, hidden_layers=[7], batch_size=32, epochs=5):
    skf = StratifiedKFold()
    skf.get_n_splits(X, y)
    sumofsquares = np.zeros([13])
    sumofweights = np.zeros([13])
    for train_index, test_index in skf.split(X, y):
        #print("TRAIN:", train_index, "TEST:", test_index)
        #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)
        model = keras.models.Sequential()
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        for i, layer in enumerate(hidden_layers):
            if i == 0:
                model.add(keras.layers.Dense(units=layer, input_dim=13, activation=activation_function))
            else:
                model.add(keras.layers.Dense(units=layer, activation=activation_function))
        model.add(keras.layers.Dense(units=1, activation=activation_function))
        model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=keras.metrics.BinaryAccuracy()) #'sgd 'keras.metrics.BinaryAccuracy()
        model.summary()
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
        print("Test Results:")
        results = model.evaluate(X_test, y_test, batch_size=batch_size)
        for i, layer_size in enumerate(hidden_layers):
            for index in range(layer_size):
                print(f"hidden_layers[{i}] = index=[{index}] {model.weights[0].numpy()[:,index]}")
                sumofsquares += model.weights[0].numpy()[:,index]**2
                sumofweights += model.weights[0].numpy()[:,index]
                #print(f"model.weights len = {len(model.weights)}")
    print("absvalues=")
    print(absvalues)
    return sumofsquares,sumofweights